<a href="https://colab.research.google.com/github/ranabilal09/Coding-Assistant/blob/main/Self_Corrective_Coding_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
!pip install --upgrade --quiet langchain langgraph langsmith langchain_community langchain_core langchain_huggingface langchain-groq langchain-anthropic langchain-google-genai langchain-chroma faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 9.9 MB/s eta 0:00:00


In [16]:
from google.colab import userdata
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = userdata.get('HF_TOKEN')
os.environ["GOOGLE_API_KEY"] = userdata.get("Gemini_Api_Key")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("langchai_api_key")
os.environ["LANGCHAIN_PROJECT"] = "Self-Corrective-Coding-Assistant"
os.environ["ANTHROPIC_API_KEY"] = userdata.get("anthropic_api_key")

In [17]:
from bs4 import BeautifulSoup as Soup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader

# LCEL docs
url = "https://python.langchain.com/docs/concepts/lcel/"
loader = RecursiveUrlLoader(
    url=url, max_depth=20, extractor=lambda x: Soup(x, "html.parser").text
)
docs = loader.load()

# Sort the list based on the URLs and get the text
d_sorted = sorted(docs, key=lambda x: x.metadata["source"])
d_reversed = list(reversed(d_sorted))
concatenated_content = "\n\n\n --- \n\n\n".join(
    [doc.page_content for doc in d_reversed]
)

In [18]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

### Anthropic

from pydantic import BaseModel, Field

# Data model
class code(BaseModel):
    """Schema for code solutions to questions about LCEL."""

    prefix: str = Field(description="Description of the problem and approach")
    imports: str = Field(description="Code block import statements")
    code: str = Field(description="Code block not including import statements")

# Prompt to enforce tool use
code_gen_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """<instructions> You are a coding assistant with expertise in LCEL, LangChain expression language. \n
    Here is the LCEL documentation:  \n ------- \n  {context} \n ------- \n Answer the user  question based on the \n
    above provided documentation. Ensure any code you provide can be executed with all required imports and variables \n
    defined. Structure your answer: 1) a prefix describing the code solution, 2) the imports, 3) the functioning code block. \n
    Invoke the code tool to structure the output correctly. </instructions> \n Here is the user question:""",
        ),
        ("placeholder", "{messages}"),
    ]
)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash-8b", temperature=0)

structured_llm = llm.with_structured_output(code, include_raw=True)


# Optional: Check for errors in case tool use is flaky
def check_gemini_output(tool_output):
    """Check for parse error or failure to call the tool"""

    # Error with parsing
    if tool_output["parsing_error"]:
        # Report back output and parsing errors
        print("Parsing error!")
        raw_output = str(tool_output["raw"].content)
        error = tool_output["parsing_error"]
        raise ValueError(
            f"Error parsing your output! Be sure to invoke the tool. Output: {raw_output}. \n Parse error: {error}"
        )

    # Tool was not invoked
    elif not tool_output["parsed"]:
        print("Failed to invoke tool!")
        raise ValueError(
            "You did not use the provided tool! Be sure to invoke the tool to structure the output."
        )
    return tool_output


# Chain with output check
code_chain_raw = (
    code_gen_prompt | structured_llm | check_gemini_output
)


def insert_errors(inputs):
    """Insert errors for tool parsing in the messages"""

    # Get errors
    error = inputs["error"]
    messages = inputs["messages"]
    messages += [
        (
            "assistant",
            f"Retry. You are required to fix the parsing errors: {error} \n\n You must invoke the provided tool.",
        )
    ]
    return {
        "messages": messages,
        "context": inputs["context"],
    }


# This will be run as a fallback chain
fallback_chain = insert_errors | code_chain_raw
N = 3  # Max re-tries
code_gen_chain_re_try = code_chain_raw.with_fallbacks(
    fallbacks=[fallback_chain] * N, exception_key="error"
)


def parse_output(solution):
    """When we add 'include_raw=True' to structured output,
    it will return a dict w 'raw', 'parsed', 'parsing_error'."""

    return solution["parsed"]


# Optional: With re-try to correct for failure to invoke tool
code_gen_chain = code_gen_chain_re_try | parse_output

In [19]:
question = "how do i built simple chatbot in LCEL?"
code_gen_chain.invoke(
    {"context": concatenated_content, "messages": [("user", question)]}
)

code(prefix="This code demonstrates a simple chatbot built using LCEL. It takes user input, greets the user, and then uses an LLM to generate a response.  Replace 'YOUR_OPENAI_API_KEY' with your actual OpenAI API key.", imports='\\"\\"\\"Import necessary libraries\\"\\"\\"\\nfrom langchain_core.runnables import RunnableSequence, RunnableLambda\\nfrom langchain.llms import OpenAI\\n', code='\\"\\"\\"Example of a simple chatbot using LCEL\\"\\"\\"\\nfrom langchain_core.runnables import RunnableSequence, RunnableLambda\\nfrom langchain.llms import OpenAI\\n\\n# Replace with your OpenAI API key\\nOPENAI_API_KEY = \\"YOUR_OPENAI_API_KEY\\"\\n\\n# Initialize the LLM\\nllm = OpenAI(openai_api_key=OPENAI_API_KEY)\\n\\ndef greet(input):\\n    return \\"Hello! How can I help you today?\\"\\n\\ndef respond(input):\\n    response = llm(input)\\n    return response\\n\\n# Define the chatbot chain\\nchatbot_chain = greet | respond\\n\\n# Example usage\\nuser_input = \\"Tell me a joke\\"\\noutput = c

In [20]:
from typing import TypedDict , Dict
from langchain_core.messages import BaseMessage

class GraphState(TypedDict):
  """
  Represents the state of our graph.

    Attributes:
        error : Binary flag for control flow to indicate whether test error was tripped
        messages : With user question, error messages, reasoning
        generation : Code solution
        iterations : Number of tries
  """

  error: str
  messages: str
  generation: str
  iterations: int

In [21]:
### Parameter

# Max tries
max_iterations = 3
# Reflect
# flag = 'reflect'
flag = "do not reflect"

### Nodes


def generate(state: GraphState):
    """
    Generate a code solution

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    error = state["error"]

    # We have been routed back to generation with an error
    if error == "yes":
        messages += [
            (
                "user",
                "Now, try again. Invoke the code tool to structure the output with a prefix, imports, and code block:",
            )
        ]

    # Solution
    code_solution = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [
        (
            "assistant",
            f"{code_solution.prefix} \n Imports: {code_solution.imports} \n Code: {code_solution.code}",
        )
    ]

    # Increment
    iterations = iterations + 1
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


def code_check(state: GraphState):
    """
    Check code

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, error
    """

    print("---CHECKING CODE---")

    # State
    messages = state["messages"]
    code_solution = state["generation"]
    iterations = state["iterations"]

    # Get solution components
    imports = code_solution.imports
    code = code_solution.code

    # Check imports
    try:
        exec(imports)
    except Exception as e:
        print("---CODE IMPORT CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the import test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # Check execution
    try:
        exec(imports + "\n" + code)
    except Exception as e:
        print("---CODE BLOCK CHECK: FAILED---")
        error_message = [("user", f"Your solution failed the code execution test: {e}")]
        messages += error_message
        return {
            "generation": code_solution,
            "messages": messages,
            "iterations": iterations,
            "error": "yes",
        }

    # No errors
    print("---NO CODE TEST FAILURES---")
    return {
        "generation": code_solution,
        "messages": messages,
        "iterations": iterations,
        "error": "no",
    }


def reflect(state: GraphState):
    """
    Reflect on errors

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation
    """

    print("---GENERATING CODE SOLUTION---")

    # State
    messages = state["messages"]
    iterations = state["iterations"]
    code_solution = state["generation"]

    # Prompt reflection

    # Add reflection
    reflections = code_gen_chain.invoke(
        {"context": concatenated_content, "messages": messages}
    )
    messages += [("assistant", f"Here are reflections on the error: {reflections}")]
    return {"generation": code_solution, "messages": messages, "iterations": iterations}


### Edges


def decide_to_finish(state: GraphState):
    """
    Determines whether to finish.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """
    error = state["error"]
    iterations = state["iterations"]

    if error == "no" or iterations == max_iterations:
        print("---DECISION: FINISH---")
        return "end"
    else:
        print("---DECISION: RE-TRY SOLUTION---")
        if flag == "reflect":
            return "reflect"
        else:
            return "generate"

In [24]:
from langgraph.graph import StateGraph,END

workflow = StateGraph(GraphState)

workflow.add_node("generate",generate)
workflow.add_node("code_check",code_check)
workflow.add_node("reflect",reflect)
workflow.add_node("decide_to_finish",decide_to_finish)

workflow.set_entry_point("generate")
workflow.add_edge("generate", "code_check")
workflow.add_conditional_edges(
    "code_check",
    decide_to_finish,
    {"relect": "reflect", "generate": "generate","end":END},
)
workflow.add_edge("reflect", "generate")
workflow.add_edge("generate", END )

graph = workflow.compile()

In [25]:
from langchain_core.messages import HumanMessage
question = "generate a code for biulding a voice assistant in LCEL ?"

solution = graph.invoke({"messages": [("user", question)], "iterations": 0, "error": ""})

---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: RE-TRY SOLUTION---
---GENERATING CODE SOLUTION---
---CHECKING CODE---
---CODE IMPORT CHECK: FAILED---
---DECISION: FINISH---


In [28]:
solution["generation"]

code(prefix="A simple voice assistant using LangChain's LLMChain", imports='from langchain.llms import OpenAI\\nfrom langchain.chains import LLMChain', code='\\"\\"\\"Building a simple voice assistant using LCEL and OpenAI\\"\\"\\"\\nfrom langchain.llms import OpenAI\\nfrom langchain.chains import LLMChain\\n\\n# Replace with your OpenAI API key\\nOPENAI_API_KEY = \\"YOUR_OPENAI_API_KEY\\"\\n\\n# Define the LLM\\nllm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)\\n\\n# Define a prompt template\\nPROMPT = \\"Respond to the following user input: \\\\n\\\\n{input}\\"\\n\\n# Create an LLMChain\\nvoice_assistant_chain = LLMChain(llm=llm, prompt=PROMPT)\\n\\n# Example usage\\nuser_input = \\"What is the capital of France?\\"\\nresponse = voice_assistant_chain.run(user_input)\\nprint(response)')